In [1]:
# This cell has a bit of initial setup. You can click the triangle to the left to expand it.
# Click the "Run" button immediately above the notebook in order to execute the contents of any cell
# WARNING: Each cell in the notebook relies upon results generated by previous cells
#   The most common problem beginners have is to execute a cell before all its predecessors
#   If you do this, you can restart the kernel (see the "Kernel" menu above) and start over
%matplotlib inline
import matplotlib.pyplot as plt

# The first step is to be able to bring things in from different directories
import sys 
import os

sys.path.insert(0, os.path.abspath('../lib'))

#from util import log_progress

import numpy as np
import HARK 
from time import clock
from copy import deepcopy
mystr = lambda number : "{:.4f}".format(number)
from HARK.utilities import plotFuncs

In [2]:
import HARK.ConsumptionSaving.ConsIndShockModel as Model        # The consumption-saving micro model
import HARK.ConsumptionSaving.ConsumerParameters as Params    # Parameters for the consumer type
from HARK.utilities import plotFuncsDer, plotFuncs              # Some tools
from time import time
mystr = lambda number : "{:.4f}".format(number)
do_simulation = True
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Define a parameter dictionary with baseline parameter values

# Set the baseline parameter values 
PermGroFac = 1.02
Rfree      = 1
DiscFac    = 0.96
CRRA       = 2.00
UnempPrb   = 0.005
IncUnemp   = 0.0
PermShkStd = 0.1
TranShkStd = 0.1
# Import default parameter values
import HARK.ConsumptionSaving.ConsumerParameters as Params 

# Make a dictionary containing all parameters needed to solve the model
base_params = Params.init_idiosyncratic_shocks

# Set the parameters for the baseline results in the paper
# using the variable values defined in the cell above
base_params['PermGroFac'] = [PermGroFac]   # Permanent income growth factor
base_params['Rfree']      = Rfree          # Interest factor on assets
base_params['DiscFac']    = DiscFac        # Time Preference Factor
base_params['CRRA']       = CRRA           # Coefficient of relative risk aversion
base_params['UnempPrb']   = UnempPrb       # Probability of unemployment (e.g. Probability of Zero Income in the paper)
base_params['IncUnemp']   = IncUnemp       # Induces natural borrowing constraint
base_params['PermShkStd'] = [PermShkStd]   # Standard deviation of log permanent income shocks
base_params['TranShkStd'] = [TranShkStd]   # Standard deviation of log transitory income shocks

# Some technical settings that are not interesting for our purposes
base_params['LivPrb']       = [1.0]   # 100 percent probability of living to next period
base_params['CubicBool']    = True    # Use cubic spline interpolation
base_params['T_cycle']      = 1       # No 'seasonal' cycles
base_params['BoroCnstArt']  = None    # No artificial borrowing constraint


In [4]:
from HARK.ConsumptionSaving.ConsIndShockModel import IndShockConsumerType
baseEx_inf = IndShockConsumerType(cycles=0,**base_params)

base_params['PermGroFac']=[1.04]
baseEx_infg= IndShockConsumerType(cycles=0,**base_params)

base_params['PermGroFac']=[1.02]
base_params['DiscFac'] = 0.9
baseEx_infd= IndShockConsumerType(cycles=0,**base_params)

baseEx_inf.solve()
baseEx_inf.unpackcFunc()

baseEx_infg.solve()
baseEx_infg.unpackcFunc()

baseEx_infd.solve()
baseEx_infd.unpackcFunc()


The given type violates the finite human wealth condition; the finite human wealth factor value 1.02000 
The given type violates the finite human wealth condition; the finite human wealth factor value 1.04000 
The given type violates the finite human wealth condition; the finite human wealth factor value 1.02000 


In [5]:
if do_simulation:
    baseEx_inf.T_sim = 100 #simulate Lifecycle_Unskilled.T_sim periods for 10000(default) agents
    baseEx_inf.track_vars = ['aNrmNow','mNrmNow','cNrmNow','pLvlNow','t_age'] #track these variables
    baseEx_inf.initializeSim()
    baseEx_inf.simulate()
    
if do_simulation:
    baseEx_infg.T_sim = 100 #simulate Lifecycle_Unskilled.T_sim periods for 10000(default) agents
    baseEx_infg.track_vars = ['aNrmNow','mNrmNow','cNrmNow','pLvlNow','t_age'] #track these variables
    baseEx_infg.initializeSim()
    baseEx_infg.simulate()

if do_simulation:
    baseEx_infd.T_sim = 100 #simulate Lifecycle_Unskilled.T_sim periods for 10000(default) agents
    baseEx_infd.track_vars = ['aNrmNow','mNrmNow','cNrmNow','pLvlNow','t_age'] #track these variables
    baseEx_infd.initializeSim()
    baseEx_infd.simulate()

In [6]:
import pandas as pd
raw_data = {'T_age': baseEx_inf.t_age_hist.flatten()+24, #add 24 to make the starting period 25. age is same for all occupations so so need to make three of these.
            'cNrm': baseEx_inf.cNrmNow_hist.flatten(),
            'logcNrm': np.log(baseEx_inf.cNrmNow_hist.flatten()),
            'pLvl': baseEx_inf.pLvlNow_hist.flatten(),
            'logpLvl': np.log(baseEx_inf.pLvlNow_hist.flatten()),
            'mNrm': baseEx_inf.mNrmNow_hist.flatten(),
            'logmNrm': np.log(baseEx_inf.mNrmNow_hist.flatten()),
            'aNrm': baseEx_inf.aNrmNow_hist.flatten(),    
            'cNrmg': baseEx_infg.cNrmNow_hist.flatten(),
            'logcNrmg': np.log(baseEx_infg.cNrmNow_hist.flatten()),
            'pLvlg': baseEx_infg.pLvlNow_hist.flatten(),
            'logpLvlg': np.log(baseEx_infg.pLvlNow_hist.flatten()),
            'mNrmg': baseEx_infg.mNrmNow_hist.flatten(),
            'logmNrmg': np.log(baseEx_infg.mNrmNow_hist.flatten()),
            'aNrmg': baseEx_infg.aNrmNow_hist.flatten(),
            'cNrmd': baseEx_infd.cNrmNow_hist.flatten(),
            'logcNrmd': np.log(baseEx_infd.cNrmNow_hist.flatten()),
            'pLvld': baseEx_infd.pLvlNow_hist.flatten(),
            'logpLvld': np.log(baseEx_infd.pLvlNow_hist.flatten()),
            'mNrmd': baseEx_infd.mNrmNow_hist.flatten(),
            'logmNrmd': np.log(baseEx_infd.mNrmNow_hist.flatten()),
            'aNrmd': baseEx_infd.aNrmNow_hist.flatten()}

#Make the simulated results into a dataset

Data = pd.DataFrame(raw_data) #make the raw data into a formal dataset
Data['Cons'] = Data.cNrm * Data.pLvl #cNrmNow has to be multiplied by pLvlNow
Data['logCons'] = Data.logcNrm + Data.logpLvl
Data['Consg'] = Data.cNrmg * Data.pLvlg #cNrmNow has to be multiplied by pLvlNow
Data['logConsg'] = Data.logcNrmg + Data.logpLvlg
Data['Consd'] = Data.cNrmd * Data.pLvld #cNrmNow has to be multiplied by pLvlNow
Data['logConsd'] = Data.logcNrmd + Data.logpLvld





Data['m'] = Data.mNrm * Data.pLvl
Data['logm'] = Data.logmNrm
Data['mg'] = Data.mNrmg * Data.pLvlg
Data['logmg'] = Data.logmNrmg
Data['md'] = Data.mNrmd * Data.pLvld
Data['logmd'] = Data.logmNrmd



Data['a'] = Data.aNrm * Data.pLvl
Data['ag'] = Data.aNrmg * Data.pLvlg
Data['ad'] = Data.aNrmd * Data.pLvld


Data['Inc'] = Data.pLvl
Data['logInc'] = Data.logpLvl
Data['Incg'] = Data.pLvlg
Data['logIncg'] = Data.logpLvlg
Data['Incd'] = Data.pLvld
Data['logIncd'] = Data.logpLvld








AgeMeans = Data.groupby(['T_age']).mean().reset_index() # Group the dataset by T_age and get the mean.

In [7]:
import math
table1 = np.zeros((3,7))

#Growth rate of aggregate consumption
table1[0,0] = math.log(AgeMeans.Cons[99]) - math.log(AgeMeans.Cons[98])

#Average growth rate of household permanent income
table1[0,1]=AgeMeans.logpLvl[99]-AgeMeans.logpLvl[98]

#Average growth rate of household consumption
table1[0,2]=AgeMeans.logCons[99]-AgeMeans.logCons[98]

#Aggregate personal saving rate
table1[0,3]=1-(AgeMeans.Cons[99]/(AgeMeans.m[99]-AgeMeans.a[98]))

#Average MPC out of wealth
table1[0,4]=(baseEx_inf.cFunc[0](AgeMeans.mNrm[99]+0.00001) - baseEx_inf.cFunc[0](AgeMeans.mNrm[99]))/0.00001

#Average Net wealth
table1[0,5]=(AgeMeans.mNrm[99] - AgeMeans.cNrm[99])

#Target net wealth
table1[0,6]=baseEx_inf.solution[0].mNrmSS - baseEx_inf.cFunc[0](baseEx_inf.solution[0].mNrmSS)

In [8]:
#Growth rate of aggregate consumption
table1[1,0]=math.log(AgeMeans.Consg[99]) - math.log(AgeMeans.Consg[98])

#Average growth rate of household permanent income
table1[1,1]=AgeMeans.logpLvlg[99]-AgeMeans.logpLvlg[98]

#Average growth rate of household consumption
table1[1,2]=AgeMeans.logConsg[99]-AgeMeans.logConsg[98]

#Aggregate personal saving rate
table1[1,3]=1-(AgeMeans.Consg[99]/(AgeMeans.mg[99]-AgeMeans.ag[98]))

#Average MPC out of wealth
table1[1,4]=(baseEx_infg.cFunc[0](AgeMeans.mNrmg[99]+0.00001) - baseEx_infg.cFunc[0](AgeMeans.mNrmg[99]))/0.00001

#Average Net wealth
table1[1,5]=(AgeMeans.mNrmg[99] - AgeMeans.cNrmg[99])

#Target net wealth
table1[1,6]=baseEx_infg.solution[0].mNrmSS - baseEx_infg.cFunc[0](baseEx_infg.solution[0].mNrmSS)

In [9]:
#Growth rate of aggregate consumption
table1[2,0] = math.log(AgeMeans.Consd[99]) - math.log(AgeMeans.Consd[98])

#Average growth rate of household permanent income
table1[2,1]=AgeMeans.logpLvld[99]-AgeMeans.logpLvld[98]

#Average growth rate of household consumption
table1[2,2]=AgeMeans.logConsd[99]-AgeMeans.logConsd[98]

#Aggregate personal saving rate
table1[2,3]=1-(AgeMeans.Consd[99]/(AgeMeans.md[99]-AgeMeans.ad[98]))

#Average MPC out of wealth
table1[2,4]=(baseEx_infd.cFunc[0](AgeMeans.mNrmd[99]+0.00001) - baseEx_infd.cFunc[0](AgeMeans.mNrmd[99]))/0.00001

#Average Net wealth
table1[2,5]=(AgeMeans.mNrmd[99] - AgeMeans.cNrmd[99])

#Target net wealth
table1[2,6]=baseEx_infd.solution[0].mNrmSS - baseEx_infd.cFunc[0](baseEx_infd.solution[0].mNrmSS)

In [11]:
import pandas as pd



# Data frame of the results we calculated
table = pd.DataFrame(table1)
table.columns = ['Growth rate of aggregate consumption', 'Average growth rate of household permanent income', 'Average growth rate of household consumption', 'Aggregate personal saving rate', 'Average MPC out of wealth','Average net wealth','Target net wealth'] # add names for columns
table.index = ['Base Value','g = .04','depreciation = .10']
table

,Growth rate of aggregate consumption,Average growth rate of household permanent income,Average growth rate of household consumption,Aggregate personal saving rate,Average MPC out of wealth,Average net wealth,Target net wealth
Base Value,0.020957,0.014807,0.014978,0.006265,0.315820,0.341770,0.313770
g = .04,0.040290,0.034225,0.034508,0.009891,0.414938,0.265945,0.246728
depreciation = .10,0.020821,0.014807,0.015143,0.004310,0.477366,0.230806,0.214614


In [ ]:
table1 = np.zeros((3,8))
table1[1,1]